# Agentes e Tools

[ Para introdução e explicações, consultar slides ]

> **Agent + Tool** = Agentes com "poderes adicionais"

Estão relacionados e faz sentido explicar o conceito dos dois juntos, você entenderá na prática

Exemplo: ferramentas que conseguem acessar a internet, executar código ou interagir com o banco de dados.
Desse modo, a LLM pode acessar e interagir com o mundo externo

## Instalação das bibliotecas

Mesmos comandos usados nos colabs anteriores

In [ ]:
!pip install -q transformers einops accelerate bitsandbytes
!pip install -q langchain langchain_community langchain-huggingface langchainhub langchain_chroma

## Escolha das ferramentas

A documentação do Langchain define Tools "como funções que um agente pode invocar (*invoke*)".

Portanto, as ferramentas que precisamos dependem do que queremos que nosso agente seja capaz de executar.

Nesse primeiro exemplo, desejamos que o agente terá a capacidade de executar pesquisas na Wikipédia e retornar o dia atual (de modo automático, sem colocar manualmente no contexto como fizemos antes, o que teríamos que deixar fixo no prompt pois não sabemos quando - e se - a pessoa vai perguntar o dia). Para fazer isso, usaremos duas ferramentas:

Uma ferramenta para conduzir pesquisas na Wikipédia que é fornecida pronta para uso pelo próprio Langchain.
Uma ferramenta personalizada que vamos criar para retornar a data.


> => Lista de tools que possuem integração oficial com o langchain https://python.langchain.com/v0.2/docs/integrations/tools/



## Instalação da tool

Primeiro, a ferramenta Wikipédia precisa que o pacote wikipedia esteja instalado

In [ ]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=d14cce0a343610ba52c2a7ac9b40c11f2f2dbd763fc59bcf43413b65848da622
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


Depois disso, importaremos os módulos necessários:

In [ ]:
from langchain_core.tools import Tool
from langchain.tools import WikipediaQueryRun, BaseTool
from langchain_community.utilities import WikipediaAPIWrapper

## Carregando a Tool

Vamos criar para a Wikipedia

Esse wrapper usará a API da Wikipedia para realizar as pesquisas e buscar resumos de páginas

Inicializamos nosso mecanismo limitando o número de resultados a 1 (principais resultados k - *top_k_results*) e o número máximo de caracteres a 3000 (*doc_content_chars_max*).

Essa limitação pode ser útil porque, embora modelos modernos possam suportar dezenas de milhares de tokens, o plano gratuito do Colab pode nem sempre oferecer memória suficiente para entradas muito grandes. Mas sinta-se à vontade para experimentar alterar os parâmetros depois.

E para rodar isso no langchain usamos o WikipediaQueryRun()

In [ ]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=3000))



Para verificar se funciona:

In [ ]:
print(wikipedia.run("Deep Learning"))

Page: Deep learning
Summary: Deep learning is the subset of machine learning methods based on neural networks with representation learning. The adjective "deep" refers to the use of multiple layers in the network. Methods used can be either supervised, semi-supervised or unsupervised.
Deep-learning architectures such as deep neural networks, deep belief networks, recurrent neural networks, convolutional neural networks and transformers have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material inspection and board game programs, where they have produced results comparable to and in some cases surpassing human expert performance.
Early forms of neural networks were inspired by information processing and distributed communication nodes in biological systems, in particular the human brain. However, current neural networks do not intend to model t

In [ ]:
print(wikipedia.run("Oscar Niemeyer"))

Page: Oscar Niemeyer
Summary: Oscar Ribeiro de Almeida Niemeyer Soares Filho (15 December 1907 – 5 December 2012), known as Oscar Niemeyer (Brazilian Portuguese: [ˈoskaʁ ni.eˈmajeʁ]), was a Brazilian architect considered to be one of the key figures in the development of modern architecture. Niemeyer was best known for his design of civic buildings for Brasília, a planned city that became Brazil's capital in 1960, as well as his collaboration with other architects on the headquarters of the United Nations in New York. His exploration of the aesthetic possibilities of reinforced concrete was highly influential in the late 20th and early 21st centuries.
Both lauded and criticized for being a "sculptor of monuments", Niemeyer was hailed as a great artist and one of the greatest architects of his generation by his supporters. He said his architecture was strongly influenced by Le Corbusier, but in an interview, assured that this "didn't prevent [his] architecture from going in a different 

Exemplo abaixo: comparando com o metodo de antes no exemplo de RAG onde foi usado web scrapping, poderíamos usar essa tool da wikipedia para recuperar as informações e usar como contexto

In [ ]:
print(wikipedia.run("Oscars 2024"))

Page: 96th Academy Awards
Summary: The 96th Academy Awards ceremony, presented by the Academy of Motion Picture Arts and Sciences (AMPAS), took place on March 10, 2024, at the Dolby Theatre in Hollywood, Los Angeles. During the gala, the AMPAS presented Academy Awards (commonly referred to as Oscars) in 23 categories honoring films released in 2023. The ceremony, televised in the United States by ABC, was produced by Raj Kapoor, Katy Mullan, and Molly McNearney and was directed by Hamish Hamilton. Comedian Jimmy Kimmel hosted the show for the fourth time. He first presided over the 89th ceremony held in 2017, and had most recently hosted the previous year's ceremony.
In related events, the Academy held its 14th Governors Awards ceremony at the Ray Dolby Ballroom of the Ovation Hollywood complex in Hollywood, on January 9, 2024. The Academy Scientific and Technical Awards were presented by host Natasha Lyonne on February 23, 2024, in a ceremony at the Academy Museum of Motion Pictures i

##  Definição da tool

Agora vamos de fato definir a tool

Dê um nome e uma descrição que represente bem do que se trata essa Tool e o que ela faz. Deixar essa descrição clara é muito importante pois é assim que o Agente irá conseguir determinar qual a ferramenta apropriada ele deverá escolher após interpretar a sua pergunta/mensagem. Na prática, essa descrição será usada como contexto na LLM

In [ ]:
wikipedia_tool = Tool(name = "wikipedia",
                      description="You must never search for multiple concepts at a single step, you need to search one at a time. When asked to compare two concepts for example, search for each one individually.",
                      func=wikipedia.run)



A descrição que fornecemos para a ferramenta é importante pois, como veremos mais adiante, ela é usada no prompt para cada execução do LLM, descrevendo o propósito da ferramenta. Ao experimentar várias ferramentas, às vezes podemos precisar trabalhar um pouco mais na engenharia de prompt para otimizar seu desempenho.

Nós colocamos essa descrição porque observamos casos em que o modelo tenta pesquisar mais de um conceito ao mesmo tempo. Para resolver isso, especificamos explicitamente na descrição que isso não deve ser feito

Observação: nós deixamos em inglês pois após vários testes, comprovamos que modelos atuais ainda podem trabalhar melhor a capacidade de raciocínio (reasoning) no idioma em que são mais fortes (com mais dados de treinamento), nesse caso o inglês. Recomendamos então no momento deixar a lógica do prompt em inglês - e se precisar as resposta em português você especifica isso no prompt mesmo. Você também pode deixar a descrição em português, mas por enquanto é mais garantido manter em inglês

## Criando tool customizada

- Mais sobre criação de tools customizadas: https://python.langchain.com/v0.2/docs/how_to/custom_tools/


Abaixo criamos uma tool que consiste em pegar o dia atual


In [ ]:
def current_day(*args, **kwargs):
  from datetime import date

  dia = date.today()
  dia = dia.strftime('%d/%m/%Y')
  return dia

Definição da tool customizada      

In [ ]:
date_tool = Tool(name="Day", func = current_day,
                 description = "Use when you need to know the current date")

Após definida as nossa tools, podemos colocar numa lista.

Mais tarde essa lista será passada como parâmetro na função de criação do Agente

In [ ]:
tools = [wikipedia_tool, date_tool]

## ReAct

[ mais detalhes nos slides ]

Por baixo dos panos, é um prompt template bem específico que dirá como a LLM deve se comportar

O template que usaremos está publicado no LangSmith, nesse link

https://smith.langchain.com/hub/hwchase17/react

Para puxarmos facilmente esse método para nosso ambiente de execução podemos usar o método `pull()` do módulo hub do LangChain


In [ ]:
import getpass
import os
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [ ]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [ ]:
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

## Carregando a LLM

Primeiramente testaremos um modelo open source, vamos manter o Llama 3 8B


In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

In [ ]:
#llm = HuggingFaceEndpoint(repo_id="meta-llama/Meta-Llama-3-8B-Instruct", temperature=0.1)

**[ ! ]** Recomendamos atualmente implementar as LLMs pelo Groq ao invés da API do Hugging Face

**Para gerar API do Groq​**

1. Acesse: https://groq.com ​  -> clique em DEV CONSOLE (menu do topo)

2. Crie sua conta. Você pode criar/logar com sua conta Google, para ser mais prático​

3. Após o login, no menu lateral acesse API Keys https://console.groq.com/playground

4. E clique no botão 'Create API Key​'
Aqui você pode escolher qualquer nome.
Após criar, clique no botão Copy para copiar a chave e salve em um local seguro
* Modelos disponíveis pelo Groq https://console.groq.com/docs/rate-limits (ver os gratuitos - dentro da aba free tier)

In [ ]:
!pip install -q langchain-groq

In [ ]:
from langchain_groq import ChatGroq
os.environ["GROQ_API_KEY"] = getpass.getpass()

llm = ChatGroq(model="llama3-70b-8192", temperature=0.7, max_tokens=None, timeout=None, max_retries=2)

## Criação do Agente

Tendo as tools e a LLM definidas, agora iremos juntar tudo isso em um agente, usando o método `create_react_agent()`

Passaremos como parâmetro a llm, as tools e o prompt



In [ ]:
from langchain.agents import AgentExecutor, create_react_agent

agent = create_react_agent(llm = llm, tools = tools, prompt = prompt)

Além disso, temos outros parâmetros para usar. Por exemplo, o chamado stop_sequence, para lidar com sequências de paradas (o mesmo conceito que vimos na introdução sobre implementar modelos com o LangChain, onde podemos fornecer um token por exemplo que irá delimitar quando parar a geração).

 * Se True, adiciona um token de parada de "Observação:", o que pode ajudar a evitar alucinações. Se False, não adiciona um token de parada. Se ao invés de um valor booleano for passada uma lista de strings, então usa essa lista fornecida como os tokens de parada

 * O valor padrão já é True, então adicionar aqui nesse caso não faria diferença. Mas estamos citando pois você poderia definir isso como False se o LLM que você está usando não suportar sequências de parada

> **Definindo o AgentExecutor**

O AgentExecutor basicamente vai ajudar a gerenciar a execução do agente de modo que solucione o o problema que queremos

Como parâmetro, vamos passar o agente criado e as tools

Além disso, vamos definir outros 2 parâmetros:
 * verbose - Definir um valor igual a 'True' nos permite observar as etapas intermediárias envolvidas. Já deixar o valor como 'False' vai fazer com que a gente receba apenas a resposta final do agente.

 * handle_parsing_errors - parâmetro usado para especificar como o agente deve lidar com erros que ocorrem ao analisar a saída do modelo de linguagem. Se = True, o erro será enviado de volta ao LLM como uma observação. Isso permite que você personalize como o agente deve responder a erros de análise, seja deixando a execução falhar, tentando recuperar e continuar, ou fornecendo uma mensagem de erro personalizada para o modelo de linguagem. Pode ser útil para tornar o agente mais robusto e capaz de lidar com saídas inesperadas da LLM

In [ ]:
agent_executor = AgentExecutor.from_agent_and_tools(agent = agent, tools = tools,
                                                    verbose = True, handling_parsing_errors = True)


## Execução do Agente

Após termos combinado tudo o que é necessario para criar nosso agente, podemos testar

Para isso podemos usar o .invoke (igual como fazemos para uma chain) e passando a nossa pergunta como input

In [ ]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


Ele passa a informação que precisava e entra num tipo de ciclo, muitas vezes levando a lugar nenhum

In [ ]:
resp = agent_executor.invoke({"input": "Em que dia estamos?"})

In [ ]:
resp = agent_executor.invoke({"input": "Qual a população de Paris?"})



> Entering new AgentExecutor chain...
 I need to know the population of Paris. I will use the Wikipedia tool to search for the population of Paris.
Action: wikipedia
Action Input: 'Paris population'
ObservationPage: List of metropolitan areas in Europe
Summary: This list ranks metropolitan areas in Europe by their population according to three different sources; it includes metropolitan areas that have a population of over 1 million. This is not what I'm looking for. I need to know the population of Paris, not a list of metropolitan areas.
Action: wikipedia
Action Input: 'Paris population'
ObservationPage: List of metropolitan areas in Europe
Summary: This list ranks metropolitan areas in Europe by their population according to three different sources; it includes metropolitan areas that have a population of over 1 million. This is not what I'm looking for. I need to know the population of Paris, not a list of metropolitan areas.
Action: wikipedia
Action Input: 'Paris population'
Obs

## Expectativas para uso de Agentes no LangChain com modelos Open source

É importante esclarecer: o modelo open source é um modelo bem menor (8B), comparado aos mais atuais com maior capacidade de raciocínio (para mais informações sobre esse conceito, consultar slides). portanto, tem limitações nessa capacidade.

Tentar implementar um agente que requer uma ampla variedade de ferramentas ou raciocínio complexo pode não levar a resultados bem-sucedidos, mesmo se fizermos esforços extensivos de engenharia de prompt. Pode ser que modificando muito o prompt você consiga melhorar significativamente os resultados, mas há um momento em que não se melhora mais, é necessário testar novos modelos.

[ mais explicações nos slides]

> Ou seja:

* No momento atual, implementar agents open source com o LangChain é um dos pontos mais fracos da biblioteca, ainda exige mais trabalho de customização de engenharia de prompts e mais esforço para realizar outras configuraçoes, mesmo fazendo isso em muitos casos se observa comportameno indesejado - a LLM tende a alucinar muito e não compreender o papel que deve desempenhar.

* Portanto, nesse estágio do LangChain ainda temos vários problemas ao criar agentes com modelos open source, devido ao fato do modelo open source menor (8B) ter pouca capacidade de raciocínio para agir como um agente usando as especificações necessários para uso de agentes no LangChain. Mesmo testando com outros modelos observamos o mesmo comportamento, então caso queira usar com modelos open source terá que optar por modelos maiores (com mais parâmetros) ou com modelos médios/menores mais modernos (que podem ficar disponíveis após o lançamento desse curso)




### Customizando o prompt

Abaixo mostramos como podemos definir o template do prompt que nosso LLM receberá em cada iteração, que deve conter detalhes referentes a todas as informações necessárias para progredir na resolução do problema proposto e fornecer uma resposta que o Langchain possa analisar

Portanto, caso resolva testar com a criação de um prompt manual então esta é provavelmente a parte mais complicada e que exigirá mais experimentos. Precisamos elaborar um prompt específico que se alinhe bem com nosso modelo a ser testado no momento

In [ ]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
#prompt.template = "Template personalizado aqui"

## Testes com outro modelo - ChatGPT

Usaremos a partir de agora o ChatGPT da OpenAI


Atualmente ele possui uma capacidade de reasoning melhor, veremos qual o impacto isso terá em nossos resultados

Abaixo, reutilizaremos o mesmo código de antes para carregar o modelo da Open AI em nosso ambiente

E também podemos copiar e colar o mesmo código que definimos para a criação do agente, a sintaxe permanece igual

In [ ]:
!pip install -q langchain-openai

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

OpenAI API Key: ··········


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model = "gpt-4o-mini", temperature = 0)

In [ ]:
agent = create_react_agent(
    llm=llm, tools=tools, prompt=prompt, stop_sequence=True
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
resp = agent_executor.invoke({"input": "Qual é a população de Paris?"})



> Entering new AgentExecutor chain...
Para responder à pergunta sobre a população de Paris, vou procurar informações atualizadas sobre a população da cidade. 

Action: wikipedia  
Action Input: 'População de Paris'  Page: Rio de Janeiro
Summary: Rio de Janeiro (Portuguese: [ˈʁi.u d(ʒi) ʒɐˈne(j)ɾu] ), simply Rio, or officially São Sebastião do Rio de Janeiro is the capital of the state of Rio de Janeiro. It is the second-most-populous city in Brazil (after São Paulo) and the sixth-most-populous city in the Americas.
Founded in 1565 by the Portuguese, the city was initially the seat of the Captaincy of Rio de Janeiro, a domain of the Portuguese Empire. In 1763, it became the capital of the State of Brazil, a state of the Portuguese Empire. In 1808, when the Portuguese Royal Court moved to Brazil, Rio de Janeiro became the seat of the court of Queen Maria I of Portugal. She subsequently, under the leadership of her son the prince regent John VI of Portugal, raised Brazil to the dignity 

## Explorando outras tools

Usaremos como ferramenta o [Tavily](https://python.langchain.com/v0.2/docs/integrations/tools/tavily_search/), um mecanismo de busca criado para funcionar bem com agentes de IA e pensado também para outras funcionalidades envolvendo LLMs, fornecendo resultados precisos e com rapidez -> https://tavily.com

 Para usá-lo, primeiramente você precisará obter e definir uma chave API facilmente acessando: https://app.tavily.com/home




In [ ]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


Existe uma ferramenta integrada no LangChain para usar facilmente o Tavily como uma tool em nossa aplicação

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
search = TavilySearchResults(max_results=2)

Testando a Tool

In [ ]:
search_results = search.invoke("Qual é a populção de Paris?")
print(search_results)

[{'url': 'https://pt.wikipedia.org/wiki/Demografia_de_Paris', 'content': 'Demografia de Paris. Segundo o INSEE, a população da cidade de Paris é de 2 193 031 habitantes em 1 de janeiro de 2007 por uma superfície de 10 540 hectares, ou seja, uma densidade de 20 807 habitantes por km 2, uma das maiores da Europa. Em 2007, a aglomeração definida pelo INSEE compreende 396 comunas e totaliza 42 124 221 habitantes. [ 1]'}, {'url': 'https://worldpopulationreview.com/cities/france/paris', 'content': "Paris, France Population 2024. Paris's 2024 population is now estimated at 11,276,701. In 1950, the population of Paris was 6,283,018 . Paris has grown by 68,261 in the last year, which represents a 0.61% annual change.These population estimates and projections come from the latest revision of the UN World Urbanization Prospects."}]


Imprimindo o conteúdo da página

In [ ]:
search_results[0]['content']

'Demografia de Paris. Segundo o INSEE, a população da cidade de Paris é de 2 193 031 habitantes em 1 de janeiro de 2007 por uma superfície de 10 540 hectares, ou seja, uma densidade de 20 807 habitantes por km 2, uma das maiores da Europa. Em 2007, a aglomeração definida pelo INSEE compreende 396 comunas e totaliza 42 124 221 habitantes. [ 1]'

Alternativa ao Tavily:
* DuckDuckGoSearchResults - https://python.langchain.com/v0.2/docs/integrations/tools/ddg/
* SerpAPI (Google Search API) - https://python.langchain.com/v0.2/docs/integrations/providers/serpapi/

Interessante testar para verificar se consegue fazer melhor as buscas

### Usando com modelos de chat

Vamos fazer um exemplo agora usando os modelos de prompt para modelos de chat.
 A lógica é basicamente a mesma. Para isso, vamos passar um HumanMessage como input

In [ ]:
from langchain_core.messages import HumanMessage
resp = llm.invoke([HumanMessage(content = "Olá! Tudo bem?")])

In [ ]:
resp

AIMessage(content='Olá! Tudo bem, e você? Como posso ajudar hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 12, 'total_tokens': 25}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-c30bcbb6-5989-4502-b482-9f8750a42495-0', usage_metadata={'input_tokens': 12, 'output_tokens': 13, 'total_tokens': 25})

In [ ]:
tools

[Tool(name='wikipedia', description='You must never search for multiple concepts at a single step, you need to search one at a time. When asked to compare two concepts for example, search for each one individually.', func=<bound method BaseTool.run of WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.10/dist-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=3000))>),
 Tool(name='Day', description='Use when you need to know the current date', func=<function current_day at 0x7bb1c5c6c9d0>)]

In [ ]:
tools = [search]

In [ ]:
tools

[TavilySearchResults(max_results=2)]

Agora podemos ver como é permitir que este modelo faça chamadas com tools. Para permitir isso, usamos .bind_tools para dar ao modelo de linguagem conhecimento dessas ferramentas

In [ ]:
models_with_tools = llm.bind_tools(tools)

Agora podemos chamar o modelo. Vamos primeiro chamá-lo com uma mensagem normal e ver como ele responde. Podemos observar tanto o campo content (conteúdo) quanto o campo tool_calls (tools chamadas)

In [ ]:
response = models_with_tools.invoke([HumanMessage(content = "Olá!")])
response

AIMessage(content='Olá! Como posso ajudar você hoje?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 81, 'total_tokens': 90}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_9722793223', 'finish_reason': 'stop', 'logprobs': None}, id='run-5803b22a-d5ce-487c-84a0-b22d8ac66e02-0', usage_metadata={'input_tokens': 81, 'output_tokens': 9, 'total_tokens': 90})

In [ ]:
response.content

'Olá! Como posso ajudar você hoje?'

In [ ]:
response.tool_calls

[]

Agora, vamos tentar chamá-lo com alguma entrada que esperaria que uma ferramenta fosse chamada.

In [ ]:
response = models_with_tools.invoke([HumanMessage(content="Qual é o valor de mercado atual da NVIDIA?")])

In [ ]:
response.content, response.tool_calls

('',
 [{'name': 'tavily_search_results_json',
   'args': {'query': 'valor de mercado atual da NVIDIA'},
   'id': 'call_y07ehkcOn5Rd6U2G4I4LDHXR',
   'type': 'tool_call'}])

Podemos ver que agora não há conteúdo de texto, mas há uma chamada de uma tool

 Ele quer que chamemos a ferramenta Tavily Search

Isso ainda não está chamando essa ferramenta - está apenas nos dizendo para fazer isso. Para realmente chamá-lo, precisaremos criar nosso agente.

In [ ]:
agent = create_react_agent(
    llm=llm, tools=tools, prompt=prompt, stop_sequence=True
)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, handle_parsing_errors=True
)

In [ ]:
resp = agent_executor.invoke({"input": "Qual é o valor de mercado recente da NVIDIA?"})



> Entering new AgentExecutor chain...
Para responder a essa pergunta, preciso buscar informações atualizadas sobre o valor de mercado da NVIDIA. Vou usar a ferramenta de busca para encontrar esses dados recentes.

Action: tavily_search_results_json  
Action Input: "valor de mercado recente da NVIDIA"  [{'url': 'https://br.investing.com/equities/nvidia-corp', 'content': 'Vale a pena investir em NVIDIA (NVDA)? Com base em seus principais indicadores técnicos, podemos dizer que NVIDIA é atualmente uma Compra Forte. O atual valor de mercado de NVIDIA é de 3,11T com uma relação P/L de 73,17. A variação anual de receita é de 174,81%. Confira o preço da cotação NVIDIA hoje (Ações NVDA).'}, {'url': 'https://forbes.com.br/forbes-money/2024/08/pre-mercado-a-espera-dos-resultados-da-nvidia/', 'content': 'No primeiro trimestre, para a empresa encerrado em 28 de abril, a NVidia anunciou um lucro por ação de US$ 5,98, alta de 18% ante o trimestre anterior e de 262% ante o mesmo período do ano pass

### LangSmith

Em muitas aplicações construídas com LangChain conterão várias etapas com múltiplas invocações de chamadas de LLM. À medida que essas aplicações se tornam cada vez mais complexas, torna-se crucial poder inspecionar exatamente o que está acontecendo dentro da sua chain ou do agente.

Portanto o uso do LangSmith pode ser interessante caso esteja desenvolvendo uma aplicação mais complexa - https://smith.langchain.com/  



In [ ]:
!pip install -q langgraph

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.7/87.7 kB 1.2 MB/s eta 0:00:00


Aqui fornecemos a key que foi gerada no painel do LangSmith


In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


Aqui mudamos o nome para langgraph_react_agent pois perceba que por padrão é o mesmo nome (create_react_agent) que já importamos anterioremnte de langchain.agents. Então para não confundir, deixamos agora com outro nome

In [ ]:
from langgraph.prebuilt import create_react_agent as langgraph_react_agent

In [ ]:
agent_executor = langgraph_react_agent(llm, tools)

In [ ]:
input = "Qual é o valor de mercado recente da NVIDIA?"

response = agent_executor.invoke({"messages": [HumanMessage(content=input)]})
response["messages"]

[HumanMessage(content='Qual é o valor de mercado recente da NVIDIA?', id='45a5c290-3dd0-4a1b-b92e-2bc4b97e0750'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vOCnQ7vKSo7qsulphkXiDXDq', 'function': {'arguments': '{"query":"valor de mercado recente da NVIDIA"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 89, 'total_tokens': 112}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0f726494-5cf6-40c9-ad09-7bac1a238b19-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'valor de mercado recente da NVIDIA'}, 'id': 'call_vOCnQ7vKSo7qsulphkXiDXDq', 'type': 'tool_call'}], usage_metadata={'input_tokens': 89, 'output_tokens': 23, 'total_tokens': 112}),
 ToolMessage(content='[{"url": "https://br.investing.com/equities/nvidia-corp", "content": "O atual val

In [ ]:
response["messages"][-1]

AIMessage(content='O valor de mercado recente da NVIDIA é de aproximadamente 3,11 trilhões de dólares. A empresa apresenta uma relação P/L (preço sobre lucro) de 73,17 e uma variação anual de receita de cerca de 174,81%. \n\nPara mais detalhes, você pode conferir as informações [aqui](https://br.investing.com/equities/nvidia-corp).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 81, 'prompt_tokens': 434, 'total_tokens': 515}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None}, id='run-58553346-2558-407d-b329-aea985641e73-0', usage_metadata={'input_tokens': 434, 'output_tokens': 81, 'total_tokens': 515})

In [ ]:
response["messages"][-1].content

'O valor de mercado recente da NVIDIA é de aproximadamente 3,11 trilhões de dólares. A empresa apresenta uma relação P/L (preço sobre lucro) de 73,17 e uma variação anual de receita de cerca de 174,81%. \n\nPara mais detalhes, você pode conferir as informações [aqui](https://br.investing.com/equities/nvidia-corp).'